In [2]:
from datasets import Dataset
import json
import torch
torch.cuda.is_available()

False

In [7]:
# Function read file train
def read_data(filePath):
    f = open(filePath,encoding='utf-8')
    fileRead = json.load(f)
    context = []
    labels = []
    for object in fileRead:
        context.append(object["context"].lower()+ "<PAD>" + object["question"].lower())
        labels.append(object["labels"])
            
    dataset = {
    "context": context,
    "labels": labels
    }
    return dataset

In [8]:
dataset = read_data("./IR_model/dataset.json")
eval_dataset = read_data("./IR_model/eval_dataset.json")

In [9]:
dataset_train = Dataset.from_dict(dataset)
dataset_eval = Dataset.from_dict(eval_dataset)

In [10]:
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering, DefaultDataCollator, TrainingArguments, Trainer, AutoModelForQuestionAnswering


# Load Tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")

# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['context'], padding='max_length', truncation=True, return_tensors="pt", max_length=128)



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [65]:
# Tokenize dataset
tokenized_dataset = dataset_train.map(tokenize, batched=True,remove_columns=["context"])
tokenized_dataset_eval = dataset_eval.map(tokenize, batched=True,remove_columns=["context"])

In [68]:
from transformers import AutoModelForSequenceClassification


# Prepare model labels - useful for inference
labels = tokenized_dataset["labels"]
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [69]:
import evaluate
import numpy as np

# Metric Id
metric = evaluate.load("f1")

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [74]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments


# Define training args
training_args = TrainingArguments(
    output_dir="./IR",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset_eval,
    compute_metrics=compute_metrics,
)

In [75]:
trainer.train()

C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
                                                
 33%|███▎      | 41/123 [09:57<17:18, 12.66s/it]

{'eval_loss': 0.7480651140213013, 'eval_f1': 0.9077332298911086, 'eval_runtime': 56.6591, 'eval_samples_per_second': 3.406, 'eval_steps_per_second': 0.441, 'epoch': 1.0}


                                                
 67%|██████▋   | 82/123 [21:01<08:32, 12.51s/it]

{'eval_loss': 0.49986615777015686, 'eval_f1': 0.9077332298911086, 'eval_runtime': 52.724, 'eval_samples_per_second': 3.661, 'eval_steps_per_second': 0.474, 'epoch': 2.0}


                                                 
100%|██████████| 123/123 [30:41<00:00, 14.97s/it]

{'eval_loss': 0.4754648506641388, 'eval_f1': 0.9077332298911086, 'eval_runtime': 49.436, 'eval_samples_per_second': 3.904, 'eval_steps_per_second': 0.506, 'epoch': 3.0}
{'train_runtime': 1841.5098, 'train_samples_per_second': 1.059, 'train_steps_per_second': 0.067, 'train_loss': 0.9797941998737615, 'epoch': 3.0}


TrainOutput(global_step=123, training_loss=0.9797941998737615, metrics={'train_runtime': 1841.5098, 'train_samples_per_second': 1.059, 'train_steps_per_second': 0.067, 'train_loss': 0.9797941998737615, 'epoch': 3.0})

In [5]:
trainer.save_model()

NameError: name 'trainer' is not defined

In [16]:
from transformers import pipeline

# load model from huggingface.co/models using our repository id
classifier = pipeline("sentiment-analysis", model="./IR", tokenizer=tokenizer)

sample = "tập quán là gì ?<PAD>trường hợp súc vật thả rông theo tập quán mà gây thiệt hại thì chủ sở hữu súc vật đó phải bồi thường theo tập quán nhưng không được trái pháp luật, đạo đức xã hội ; điều 603 khoản 4."


pred = classifier(sample)
print(pred)
# [{'label': 'card_arrival', 'score': 0.9903606176376343}]

[{'label': 1, 'score': 0.978694498538971}]
